In [ ]:
print('Setup complete.')

# Plan-Do-Check Demo

## Learning Objectives
- Implement systematic planning workflows with AI
- Build validation loops for AI-generated content
- Create feedback mechanisms for iterative improvement
- Design robust multi-step AI workflows

## The Plan-Do-Check Pattern

This demo shows how to build reliable AI workflows using the classic quality management cycle:
1. **Plan** - Generate a structured plan for the task
2. **Do** - Execute the plan step by step
3. **Check** - Validate results and identify improvements
4. **Act** - Iterate based on feedback (bonus: not in this demo)

In [ ]:
# Install required packages
!pip install asksageclient pip_system_certs rich pydantic tiktoken

In [ ]:
# ================================
# 🔐 Cell 1 — Load secrets (Colab) + pricing + token utils
# ================================
import os, time, csv
from typing import Optional, Dict
import tiktoken

from google.colab import userdata

ASKSAGE_API_KEY = userdata.get("ASKSAGE_API_KEY")
ASKSAGE_BASE_URL = userdata.get("ASKSAGE_BASE_URL")
ASKSAGE_EMAIL = userdata.get("ASKSAGE_EMAIL")

assert ASKSAGE_API_KEY, "ASKSAGE_API_KEY not provided."
assert ASKSAGE_EMAIL, "ASKSAGE_EMAIL not provided."

print("✓ Secrets loaded")
print("  • EMAIL:", ASKSAGE_EMAIL)
print("  • BASE URL:", ASKSAGE_BASE_URL or "(default)")

# Pricing (USD per 1,000,000 tokens)
PRICES_PER_M = {
    "gpt-5": {"input_per_m": 1.25, "output_per_m": 10.00},
    "gpt-5-mini": {"input_per_m": 0.25, "output_per_m": 2.00},
}

# Tokenizer
enc = tiktoken.get_encoding("o200k_base")

def count_tokens(text: str) -> int:
    return len(enc.encode(text or ""))

def cost_usd(model: str, input_tokens: int, output_tokens: int) -> float:
    if model not in PRICES_PER_M:
        raise ValueError(f"Unknown model: {model}")
    r = PRICES_PER_M[model]
    return (input_tokens / 1_000_000) * r["input_per_m"] + (output_tokens / 1_000_000) * r["output_per_m"]

In [ ]:
# ================================
# 🔧 Cell 2 — Import bootcamp_common and setup AskSage client
# ================================
import sys
sys.path.append('../../../')  # Adjust path to reach bootcamp_common

from bootcamp_common.ask_sage import AskSageClient

# Initialize AskSage client
client = AskSageClient(
    api_key=ASKSAGE_API_KEY,
    base_url=ASKSAGE_BASE_URL
)

print("✓ AskSage client initialized")

In [ ]:
import os
import json
import time
from typing import Dict, List, Optional, Any
from dataclasses import dataclass
from datetime import datetime
from enum import Enum

import openai
from rich.console import Console
from rich.panel import Panel
from rich.table import Table
from rich.progress import Progress, SpinnerColumn, TextColumn
from pydantic import BaseModel

console = Console()
print("✅ Libraries loaded successfully")

## Define Workflow Schemas

In [ ]:
class TaskStep(BaseModel):
    """Individual step in a plan"""
    step_number: int
    description: str
    estimated_time: int  # minutes
    dependencies: List[int]  # step numbers this depends on
    success_criteria: str

class ExecutionPlan(BaseModel):
    """Complete execution plan"""
    goal: str
    steps: List[TaskStep]
    total_estimated_time: int
    risks: List[str]
    success_metrics: List[str]

class ExecutionResult(BaseModel):
    """Result of executing a step"""
    step_number: int
    output: str
    success: bool
    actual_time: int  # minutes
    notes: str

class QualityCheck(BaseModel):
    """Quality assessment of results"""
    criteria: str
    score: int  # 1-10
    issues: List[str]
    improvements: List[str]
    overall_assessment: str

class PlanDoCheckWorkflow:
    """Implementation of Plan-Do-Check workflow"""
    
    def __init__(self):
        self.setup_client()
        self.current_plan: Optional[ExecutionPlan] = None
        self.execution_results: List[ExecutionResult] = []
        self.quality_checks: List[QualityCheck] = []
    
    def setup_client(self):
        """Setup API client with fallback to mock"""
        if os.getenv('OPENAI_API_KEY'):
            try:
                self.client = openai.OpenAI()
                self.has_api = True
                console.print("✅ OpenAI client configured")
            except Exception as e:
                self.has_api = False
                console.print(f"⚠️ Using mock responses: {e}")
        else:
            self.has_api = False
            console.print("💡 No API key found, using mock responses")
    
    def generate_plan(self, goal: str) -> ExecutionPlan:
        """PLAN: Generate structured execution plan"""
        
        prompt = f"""Create a detailed execution plan for this goal: {goal}

Generate a plan with:
- 3-5 specific, actionable steps
- Time estimates for each step
- Dependencies between steps
- Success criteria for each step
- Overall risks and success metrics

Respond with valid JSON matching this schema:
{{
  "goal": "string",
  "steps": [
    {{
      "step_number": 1,
      "description": "string",
      "estimated_time": 30,
      "dependencies": [],
      "success_criteria": "string"
    }}
  ],
  "total_estimated_time": 120,
  "risks": ["string"],
  "success_metrics": ["string"]
}}"""
        
        if self.has_api:
            response = self.client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": prompt}],
                max_tokens=800,
                temperature=0.3
            )
            response_text = response.choices[0].message.content
        else:
            # Mock plan for blog post creation
            response_text = '''{
  "goal": "Create a comprehensive blog post about sustainable technology",
  "steps": [
    {
      "step_number": 1,
      "description": "Research current sustainable technology trends and gather credible sources",
      "estimated_time": 45,
      "dependencies": [],
      "success_criteria": "At least 5 credible sources from recent publications"
    },
    {
      "step_number": 2,
      "description": "Create detailed outline with main sections and key points",
      "estimated_time": 30,
      "dependencies": [1],
      "success_criteria": "Clear outline with 4-5 main sections and subpoints"
    },
    {
      "step_number": 3,
      "description": "Write first draft focusing on content and flow",
      "estimated_time": 90,
      "dependencies": [2],
      "success_criteria": "1500+ word draft covering all outline points"
    },
    {
      "step_number": 4,
      "description": "Edit for clarity, engagement, and SEO optimization",
      "estimated_time": 60,
      "dependencies": [3],
      "success_criteria": "Polished post with good readability score and SEO elements"
    }
  ],
  "total_estimated_time": 225,
  "risks": ["Limited access to latest research", "Topic might be too broad", "SEO competition is high"],
  "success_metrics": ["Post reaches 1500+ words", "Includes 5+ credible sources", "Readability score above 60", "At least 3 actionable insights for readers"]
}'''
        
        # Parse and validate
        try:
            plan_data = json.loads(response_text.strip().replace('```json', '').replace('```', ''))
            self.current_plan = ExecutionPlan(**plan_data)
            return self.current_plan
        except Exception as e:
            console.print(f"[red]Plan generation failed: {e}[/red]")
            raise
    
    def execute_step(self, step_number: int) -> ExecutionResult:
        """DO: Execute a specific step"""
        
        if not self.current_plan:
            raise ValueError("No plan available for execution")
        
        step = next((s for s in self.current_plan.steps if s.step_number == step_number), None)
        if not step:
            raise ValueError(f"Step {step_number} not found in plan")
        
        prompt = f"""Execute this step from our plan:

Goal: {self.current_plan.goal}
Step {step_number}: {step.description}
Success Criteria: {step.success_criteria}

Provide a realistic simulation of executing this step. Include:
- What specific actions you would take
- What deliverables/outputs you would create
- Any challenges or observations
- How you know you've met the success criteria

Keep response focused and practical (200-300 words)."""
        
        start_time = time.time()
        
        if self.has_api:
            response = self.client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": prompt}],
                max_tokens=400,
                temperature=0.5
            )
            output = response.choices[0].message.content
        else:
            # Mock execution results
            mock_outputs = {
                1: "Researched sustainable tech trends. Found 6 credible sources including IEEE papers on renewable energy storage, MIT reports on carbon capture, and industry analyses from McKinsey. Key trends identified: battery recycling, green hydrogen, sustainable AI computing. All sources are from 2023-2024.",
                2: "Created comprehensive outline: 1) Introduction to sustainable tech importance, 2) Current breakthrough technologies (batteries, renewables, AI efficiency), 3) Industry adoption challenges and solutions, 4) Future outlook and investment trends, 5) Actionable steps for businesses. Each section has 3-4 key subpoints.",
                3: "Drafted 1,650-word blog post covering all outline sections. Strong introduction hooks readers with climate urgency. Technical sections balanced with real-world examples. Included case studies from Tesla, Google, and Nordic energy companies. Clear transitions between sections maintain flow.",
                4: "Edited for clarity and SEO. Improved readability score to 65. Added target keywords naturally. Created compelling meta description. Added subheadings for better scanning. Included call-to-action encouraging readers to assess their own sustainability practices."
            }
            output = mock_outputs.get(step_number, "Step executed successfully.")
        
        actual_time = int((time.time() - start_time) * 60)  # Convert to minutes
        if actual_time < 1:
            actual_time = step.estimated_time  # Use estimate for demo
        
        result = ExecutionResult(
            step_number=step_number,
            output=output,
            success=True,  # Assume success for demo
            actual_time=actual_time,
            notes=f"Completed step {step_number} successfully"
        )
        
        self.execution_results.append(result)
        return result
    
    def quality_check(self, criteria: str) -> QualityCheck:
        """CHECK: Evaluate quality of execution results"""
        
        if not self.execution_results:
            raise ValueError("No execution results to check")
        
        # Compile all outputs for assessment
        all_outputs = "\n\n".join([f"Step {r.step_number}: {r.output}" for r in self.execution_results])
        
        prompt = f"""Evaluate the quality of these execution results against the criteria:

Criteria: {criteria}

Execution Results:
{all_outputs}

Provide assessment in this JSON format:
{{
  "criteria": "{criteria}",
  "score": 8,
  "issues": ["specific issue 1", "specific issue 2"],
  "improvements": ["suggestion 1", "suggestion 2"],
  "overall_assessment": "Brief summary of quality and recommendations"
}}

Score 1-10 where 10 is excellent quality."""
        
        if self.has_api:
            response = self.client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": prompt}],
                max_tokens=300,
                temperature=0.3
            )
            response_text = response.choices[0].message.content
        else:
            # Mock quality check
            response_text = '''{
  "criteria": "Blog post completeness and quality",
  "score": 8,
  "issues": ["Could benefit from more specific data/statistics", "Missing direct quotes from industry experts"],
  "improvements": ["Add 2-3 concrete statistics to strengthen arguments", "Include expert quotes for credibility", "Consider adding visual elements or diagrams"],
  "overall_assessment": "Strong execution with comprehensive research and clear structure. Content meets word count and covers all planned sections. Minor improvements needed for expert credibility and data support."
}'''
        
        try:
            check_data = json.loads(response_text.strip().replace('```json', '').replace('```', ''))
            quality_check = QualityCheck(**check_data)
            self.quality_checks.append(quality_check)
            return quality_check
        except Exception as e:
            console.print(f"[red]Quality check failed: {e}[/red]")
            raise

# Initialize workflow
workflow = PlanDoCheckWorkflow()
print("🔄 Plan-Do-Check workflow ready!")

## Demo: Complete Plan-Do-Check Cycle

In [ ]:
# Run complete Plan-Do-Check cycle
console.print("\n🎯 [bold blue]Running Complete Plan-Do-Check Workflow[/bold blue]\n")
console.print("─" * 60)

# PLAN Phase
console.print("\n📋 [bold yellow]PLAN PHASE[/bold yellow]")
goal = "Create a comprehensive blog post about sustainable technology trends"
console.print(f"[cyan]Goal:[/cyan] {goal}")

with Progress(SpinnerColumn(), TextColumn("[progress.description]{task.description}")) as progress:
    task = progress.add_task("Generating execution plan...", total=None)
    plan = workflow.generate_plan(goal)
    progress.update(task, completed=100)

console.print("\n[green]✅ Plan Generated![/green]")
console.print(Panel(f"**Goal:** {plan.goal}\n\n**Steps:** {len(plan.steps)}\n**Estimated Time:** {plan.total_estimated_time} minutes\n**Key Risks:** {', '.join(plan.risks[:2])}", 
                   title="Execution Plan", border_style="green"))

# Show detailed steps
steps_table = Table(title="Detailed Step Breakdown")
steps_table.add_column("#")
steps_table.add_column("Description")
steps_table.add_column("Time")
steps_table.add_column("Success Criteria")

for step in plan.steps:
    steps_table.add_row(
        str(step.step_number),
        step.description[:50] + "..." if len(step.description) > 50 else step.description,
        f"{step.estimated_time}m",
        step.success_criteria[:40] + "..." if len(step.success_criteria) > 40 else step.success_criteria
    )

console.print(steps_table)

In [ ]:
# DO Phase - Execute steps
console.print("\n🔨 [bold yellow]DO PHASE[/bold yellow]")

for step in plan.steps[:3]:  # Execute first 3 steps for demo
    console.print(f"\n[cyan]Executing Step {step.step_number}:[/cyan] {step.description}")
    
    with Progress(SpinnerColumn(), TextColumn("[progress.description]{task.description}")) as progress:
        task = progress.add_task(f"Working on step {step.step_number}...", total=None)
        result = workflow.execute_step(step.step_number)
        progress.update(task, completed=100)
    
    status = "[green]✅ Success[/green]" if result.success else "[red]❌ Failed[/red]"
    console.print(f"{status} - Completed in {result.actual_time} minutes")
    console.print(Panel(result.output[:200] + "..." if len(result.output) > 200 else result.output, 
                       title=f"Step {step.step_number} Output", border_style="blue"))

console.print(f"\n[green]✅ Executed {len(workflow.execution_results)} steps successfully![/green]")

In [ ]:
# CHECK Phase - Quality assessment
console.print("\n🔍 [bold yellow]CHECK PHASE[/bold yellow]")

criteria = "Blog post completeness, quality of research, and adherence to success metrics"
console.print(f"[cyan]Quality Criteria:[/cyan] {criteria}")

with Progress(SpinnerColumn(), TextColumn("[progress.description]{task.description}")) as progress:
    task = progress.add_task("Conducting quality assessment...", total=None)
    quality_check = workflow.quality_check(criteria)
    progress.update(task, completed=100)

# Display quality results
score_color = "green" if quality_check.score >= 8 else "yellow" if quality_check.score >= 6 else "red"
console.print(f"\n[{score_color}]📊 Quality Score: {quality_check.score}/10[/{score_color}]")

console.print(Panel(quality_check.overall_assessment, title="Quality Assessment", border_style=score_color))

if quality_check.issues:
    console.print("\n[red]🚨 Issues Identified:[/red]")
    for issue in quality_check.issues:
        console.print(f"  • {issue}")

if quality_check.improvements:
    console.print("\n[yellow]💡 Suggested Improvements:[/yellow]")
    for improvement in quality_check.improvements:
        console.print(f"  • {improvement}")

print("\n🎯 Plan-Do-Check cycle complete!")

## Workflow Summary & Analysis

In [ ]:
# Generate workflow summary
console.print("\n📊 [bold blue]Workflow Performance Summary[/bold blue]")

# Performance metrics table
metrics_table = Table(title="Execution Metrics")
metrics_table.add_column("Metric")
metrics_table.add_column("Planned")
metrics_table.add_column("Actual")
metrics_table.add_column("Variance")

total_planned_time = sum(step.estimated_time for step in plan.steps[:3])
total_actual_time = sum(result.actual_time for result in workflow.execution_results)
time_variance = ((total_actual_time - total_planned_time) / total_planned_time * 100) if total_planned_time > 0 else 0

metrics_table.add_row(
    "Total Time",
    f"{total_planned_time} min",
    f"{total_actual_time} min",
    f"{time_variance:+.1f}%"
)

metrics_table.add_row(
    "Steps Completed",
    f"{len(plan.steps)} planned",
    f"{len(workflow.execution_results)} done",
    f"{len(workflow.execution_results)/len(plan.steps)*100:.0f}%"
)

metrics_table.add_row(
    "Success Rate",
    "100%",
    f"{sum(1 for r in workflow.execution_results if r.success)/len(workflow.execution_results)*100:.0f}%",
    "On target"
)

console.print(metrics_table)

# Final assessment
if quality_check.score >= 8:
    assessment = "[green]🎉 Excellent execution! Workflow delivered high-quality results.[/green]"
elif quality_check.score >= 6:
    assessment = "[yellow]✅ Good execution with room for improvement.[/yellow]"
else:
    assessment = "[red]⚠️ Execution needs significant improvement.[/red]"

console.print(f"\n{assessment}")
console.print(f"\n[bold]Next Steps:[/bold] {'Implement suggested improvements and re-run check phase' if quality_check.improvements else 'Proceed to deployment/delivery'}")

print("\n🔄 Complete Plan-Do-Check workflow demonstrated!")

## Key Takeaways: Systematic AI Workflows

### 🔄 **The Plan-Do-Check Pattern**

1. **PLAN**: Generate structured, validated execution plans
2. **DO**: Execute steps systematically with monitoring
3. **CHECK**: Assess quality against defined criteria
4. **ACT**: Iterate based on feedback (bonus round!)

### 🎯 **Benefits of Systematic Workflows**

- **Reliability**: Consistent results through structured processes
- **Visibility**: Clear progress tracking and metrics
- **Quality**: Built-in validation and improvement loops
- **Scalability**: Repeatable patterns for complex tasks
- **Learning**: Captured insights for future improvements

### 📊 **Production Implementation Tips**

- **Validate Plans**: Check for logical dependencies and realistic timelines
- **Monitor Execution**: Track progress, time, and resource usage
- **Automate Checks**: Use quantitative metrics where possible
- **Capture Feedback**: Log lessons learned for process improvement
- **Handle Failures**: Implement fallback strategies and error recovery

### 🚀 **Advanced Extensions**

- **Multi-Agent**: Different agents for planning, execution, and checking
- **Parallel Execution**: Run independent steps concurrently
- **Dynamic Replanning**: Adjust plans based on intermediate results
- **Risk Assessment**: Proactive identification and mitigation
- **Performance Learning**: Improve estimates based on historical data

## Next: Day 2 Labs

Ready to put these patterns into practice? The labs will have you build your own prompt library, structured output pipelines, and diagnostic dashboards!